In [1]:

import numpy as np, pandas as pd
from scipy.optimize import minimize

# --- charge tes données standardisées existantes ---
sn  = pd.read_csv("data/sn.csv")   # colonnes: z, mu, mu_err
bao = pd.read_csv("data/bao.csv")  # colonnes: z, dv_rd, dv_rd_err
hz  = pd.read_csv("data/hz.csv")   # colonnes: z, H, H_err

c = 299_792.458  # km/s

def integrate_two_layer(z_grid, H0, Om_vis0, Om_som0, w0=-1.0, wa=0.0, xi=0.0):
    """
    Deux fluides: 'vis' (w~0, dust) et 'som' (w(z)=w0+wa*(1-a)).
    Transfert: Q_vis = + xi H rho_vis ; Q_som = -Q_vis.
    Intégration en z (trapèzes). Retourne rho_vis(z), rho_som(z), H(z).
    """
    z = np.asarray(z_grid)
    a = 1.0/(1.0+z)
    # normalisation densités à z=0: rho_crit0 = 3 H0^2 / (8πG) -> on travaille en unités d'Omega * rho_crit0
    rho_vis = np.zeros_like(z); rho_som = np.zeros_like(z)
    rho_vis[0] = Om_vis0
    rho_som[0] = Om_som0

    def w_of_z(zz):
        aa = 1.0/(1.0+zz)
        return w0 + wa*(1.0-aa)

    H = np.zeros_like(z)
    H[0] = H0*np.sqrt(rho_vis[0] + rho_som[0])  # (on ignore rad/curv ici; on peut ajouter si besoin)

    for i in range(1, len(z)):
        zi, zi_1 = z[i], z[i-1]
        Hi_1 = H[i-1]
        # pas en z
        dz = zi - zi_1
        # updates par Euler semi-implicite
        wsom = w_of_z(zi_1)
        Qvis  =  xi * Hi_1 * rho_vis[i-1]
        # d rho / dz = 3(1+w)/(1+z) rho - Q/((1+z)H)
        dri_vis = (3.0/(1.0+zi_1))*rho_vis[i-1] - Qvis/((1.0+zi_1)*Hi_1)
        dri_som = (3.0*(1.0+wsom)/(1.0+zi_1))*rho_som[i-1] + Qvis/((1.0+zi_1)*Hi_1)

        rho_vis[i] = max(1e-30, rho_vis[i-1] + dz*dri_vis)
        rho_som[i] = max(1e-30, rho_som[i-1] + dz*dri_som)
        H[i] = H0*np.sqrt(rho_vis[i] + rho_som[i])

    return rho_vis, rho_som, H

def DL_from_H(z, H):
    """Distance de luminosité avec H(z) discret (trapèzes) – géométrie plate."""
    z = np.asarray(z)
    Dc = np.zeros_like(z)
    for i in range(1,len(z)):
        zz = z[:i+1]
        Hz = H[:i+1]
        Dc[i] = np.trapz(1.0/Hz, zz) * c  # Mpc si H en km/s/Mpc
    DL = (1.0+z)*Dc
    return DL

def chi2_all(theta):
    # theta = [H0, Om_vis0, Om_som0, w0, wa, xi, M] ; avec contraintes minimales
    H0, Omv, Oms, w0, wa, xi, M = theta
    if not(50<H0<85 and 0.0<=Omv<=1.0 and 0.0<=Oms<=1.0 and 0.9<=Omv+Oms<=1.1 and -1.5<w0<=-0.3 and -1.5<wa<1.5 and 0.0<=xi<0.5 and -20.5<M<-18.0):
        return 1e30

    # grille z unifiée pour SN/BAO/H(z)
    z_grid = np.unique(np.concatenate([sn['z'].values, bao['z'].values, hz['z'].values, np.linspace(0,2.5,400)]))
    z_grid.sort()
    _, _, H = integrate_two_layer(z_grid, H0, Omv, Oms, w0=w0, wa=wa, xi=xi)

    # Interpolations simples
    def interp(x, xp, fp):
        return np.interp(x, xp, fp)

    # SN
    DL = DL_from_H(z_grid, H)
    mu_model = 5*np.log10(interp(sn['z'].values, z_grid, DL)) + 25 + M
    chi2_sn = np.sum(((sn['mu'].values - mu_model)/sn['mu_err'].values)**2)

    # BAO (DV/rd ~ approx avec H et Dc; simple proxy)
    Dc = np.zeros_like(z_grid)
    for i in range(1,len(z_grid)):
        Dc[i] = np.trapz(1.0/H[:i+1], z_grid[:i+1]) * c
    DV = ( (c* z_grid * (Dc**2)/H)**(1.0/3.0) )
    DVm = interp(bao['z'].values, z_grid, DV)
    # On suppose r_d fixé (147.09 Mpc) ici pour l’exemple :
    dv_rd_model = DVm / 147.09
    chi2_bao = np.sum(((bao['dv_rd'].values - dv_rd_model)/bao['dv_rd_err'].values)**2)

    # H(z)
    Hm = interp(hz['z'].values, z_grid, H)
    chi2_hz = np.sum(((hz['H'].values - Hm)/hz['H_err'].values)**2)

    return chi2_sn + chi2_bao + chi2_hz

# --- Fit rapide (point de départ) ---
x0 = [68.0, 0.30, 0.70, -1.0, 0.0, 0.02, -19.3]
bounds = [(55,85),(0,1),(0,1),(-1.5,-0.3),(-1.5,1.5),(0,0.3),(-20.5,-18.0)]
res = minimize(chi2_all, x0, method="L-BFGS-B", bounds=bounds)
print("STATUT:", res.success, res.message)
print("θ*  =", res.x)
print("χ²* =", res.fun)

FileNotFoundError: [Errno 2] No such file or directory: 'data/sn.csv'